The motivation of the experiments is to populate comparsion study for the impact of pdf versus text to GPT.
- inputs consists of text, pdf varied from token length, 10, 20, 30k, full length.
- GPT agents consist of chatgpt and GPT4

Base assumption is that results stay close with each other.

In [ ]:
# pdf
import fitz  # PyMuPDF
from PyPDF2 import PdfReader
import pymupdf
# string parse

# langchain
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
openai_api_key = ''
# dataframe processor
import numpy as np
# os
import os, requests, sys, pathlib
import time
import pandas as pd
from tqdm import tqdm


##

## GPT (FULL TEXT, 10K, 30K)

### preprocess for groundtruths, relevant result page parsing

In [ ]:
fn = '/Users/dawn.duan/Library/CloudStorage/OneDrive-CanadianTire/Documents/tetris/ivado_or/tetris-api-worker/optimization/local_experiments/aer1810/10_benchmark_datasets/groundtruth_table.csv'
groundtruth_df = pd.read_csv(fn)
# groundtruth_df

find_fname = lambda pdf_path: str(pdf_path).split('/')[-1]

def extract_text_from_pdf_as_dict(pdf_path):
    """
    Extract text from a PDF at the specified path using PyMuPDF.
    """
    with fitz.open(pdf_path) as doc:
        return {
            i+1:page.get_text() # starting from 1
            for i,page in enumerate(doc)
        }


key_indices = groundtruth_df.page_key.values
accuracies = groundtruth_df['Top-5 Accuracy'].values

# first key is file nm; second is page number for answer
ground_truths = {
    idx: [find_fname(path), key_indices[idx]]
    for idx, path in enumerate(sorted_pdf_paths)
}
ground_truths
pdf_keys = {
    path : key_indices[idx]
    for idx, path in enumerate(sorted_pdf_paths)
}
pdf_page_lengths = {
    idx: [path, key_indices[idx], max(extract_text_from_pdf_as_dict(path))]
    for idx, path in enumerate(sorted_pdf_paths)
}


# pdf_page_lengths
sorted_pdf_key_page = {
    idx: [path, key_indices[idx], extract_text_from_pdf_as_dict(path)[key_indices[idx]] if key_indices[idx]>0 else '']
    for idx, path in enumerate(sorted_pdf_paths)
}

# sorted_pdf_key_page



In [ ]:
folder_path_20 = '/Users/dawn.duan/Library/CloudStorage/OneDrive-CanadianTire/Documents/tetris/ivado_or/tetris-api-worker/optimization/local_experiments/aer1810/10_benchmark_datasets/all_papers_20/'
sorted_pdf_paths_20 = sorted([path for path in Path(folder_path_20).iterdir() if is_pdf(path)])
sorted_pdf_key_page_20 = {
        idx: [find_fname(path), path]
        for idx, path in enumerate(sorted_pdf_paths_20)
    }
df_20 = pd.DataFrame.from_dict(
    sorted_pdf_key_page_20,orient='index',
    columns=['file_name', 'path',]
)
from functools import reduce
fn = '/Users/dawn.duan/Library/CloudStorage/OneDrive-CanadianTire/Documents/tetris/ivado_or/tetris-api-worker/optimization/local_experiments/aer1810/10_benchmark_datasets/groundtruth_table.csv'
groundtruth_df = pd.read_csv(fn)
# groundtruth_df
template = groundtruth_df.copy()[['file_name', 'Paper Name', 'Model', 'Top-1 Accuracy']]
reduce(lambda l, r: pd.merge(l, r, on=['file_name'],how='right'), [template, df_20])




folder_path_100 = '/Users/dawn.duan/Library/CloudStorage/OneDrive-CanadianTire/Documents/tetris/ivado_or/tetris-api-worker/optimization/local_experiments/aer1810/10_benchmark_datasets/all_papers_100/'
sorted_pdf_paths_100 = sorted([path for path in Path(folder_path_100).iterdir() if is_pdf(path)])
sorted_pdf_key_page_100 = {
        idx: [find_fname(path), path]
        for idx, path in enumerate(sorted_pdf_paths_100)
    }
df_100 = pd.DataFrame.from_dict(
    sorted_pdf_key_page_100,orient='index',
    columns=['file_name', 'path',]
)
reduce(lambda l, r: pd.merge(l, r, on=['file_name'],how='outer'), [template, df_100])



,file_name,Paper Name,Model,Top-1 Accuracy,path
0,1312.6229v4.pdf,NaN,NaN,NaN,/Users/dawn.duan/Library/CloudStorage/OneDrive...
1,1406.2732v1.pdf,NaN,NaN,NaN,/Users/dawn.duan/Library/CloudStorage/OneDrive...
2,1412.0296v1.pdf,NaN,NaN,NaN,/Users/dawn.duan/Library/CloudStorage/OneDrive...
3,1412.6598v2.pdf,NaN,NaN,NaN,/Users/dawn.duan/Library/CloudStorage/OneDrive...
4,1502.03167v3.pdf,NaN,NaN,NaN,/Users/dawn.duan/Library/CloudStorage/OneDrive...
...,...,...,...,...,...
115,Yan_HD-CNN_Hierarchical_Deep_ICCV_2015_paper.pdf,HD-CNN: Hierarchical Deep Convolutional Neural...,HD-CNN,37.92,NaN
116,Zhang_Compact_Representation_for_2014_CVPR_pap...,Compact Representation for Image Classificatio...,NaN,-,NaN
117,Zhang_Supplementary_Meta-Learning_Towards_ICCV...,Supplementary Meta-Learning: Towards a Dynamic...,MLNN,-,NaN
118,Zhao_Saliency_Detection_by_2015_CVPR_paper.pdf,Saliency Detection by Multi-Context Deep Learn...,NaN,-,NaN


In [ ]:
# preprocessing
import regex as re

def clean_accuracy(v: str)->str:
    return v.strip('0%')

def find_groundtruth_accuracy_in_page(idx, pdf_path, rel_page):
    accuracy = clean_accuracy(accuracies[idx])
    found_accuracy_in = accuracy in rel_page
    found_accuracy_re = re.search(pattern=accuracy, string=rel_page)
    return idx, find_fname(pdf_path), accuracy, found_accuracy_in, found_accuracy_re, rel_page


pd.DataFrame(data=[find_groundtruth_accuracy_in_page(idx, pdf_path, rel_page) for idx, (pdf_path, rel_key, rel_page) in sorted_pdf_key_page.items()],
             columns=['file_idx', 'file_name', 'accuracy', 'found_accuracy_in', 'found_accu_re', 'rel_page',])

,file_idx,file_name,accuracy,found_accuracy_in,found_accu_re,rel_page
0,0,1512.03385v1.pdf,95.51,False,None,model\ntop-1 err.\ntop-5 err.\nVGG-16 [41]\n28...
1,1,1703.09844v5.pdf,-,True,"<regex.Match object; span=(79, 80), match='-'>",Published as a conference paper at ICLR 2018\n...
2,2,1803.00942v3.pdf,-,True,"<regex.Match object; span=(185, 186), match='-'>",Deep Learning with Importance Sampling\n0\n500...
3,3,1807.10108v5.pdf,-,True,"<regex.Match object; span=(60, 61), match='-'>",6\n(a) Gaussian white noise\n(b) Gaussian colo...
4,4,1807.10119v3.pdf,8,True,"<regex.Match object; span=(2569, 2570), match=...",0\n50\n100\n0\n50\n100\n150\n200\n250\n(a)\n0\...
5,5,1807.11164v1.pdf,-,True,"<regex.Match object; span=(18, 19), match='-'>",13\nModel\nFLOPs Top-1 err. (%)\nShuﬄeNet v2-5...
6,6,1807.11254v2.pdf,78.07,False,None,Extreme Network Compression via Filter Group A...
7,7,1807.11459v1.pdf,-,False,None,
8,8,1807.11626v3.pdf,-,True,"<regex.Match object; span=(104, 105), match='-'>",7.2. Disentangling Search Space and Reward\nTo...
9,9,1909.11155v1.pdf,93.03,True,"<regex.Match object; span=(608, 613), match='9...",Table 1. Classiﬁcation accuracy on CIFAR (ResN...


## Chunk methods

## Results with increased paper size (12, 20, 100)


In [ ]:
prompt_self_veri = """
extract the top1 accuracy of ImageNet from the given text and return both the sentence containing the accuracy.
Answer in a number, eg. 90.2% and the accuracy value in 1 number. 404 if it's not mentioned. Use the examples below as a guide.

Example 1:
Expected Output:
Sentence: "a table showing the Top-1 and Top-5 classification accuracy using a binarized ResNet-18 on Imagenet for various ways of constructing the scaling factor. The method "Case 4: α ⊗β ⊗γ" achieved a Top-1 accuracy of 57.1."
Accuracy: 57.1

Example 2:
Expected Output:
Sentence: "In our experiments, the model reported a top-1 accuracy of 82.1% on Imagenet under the given conditions."
Accuracy: 82.1

Example 3:
Expected Output:
Sentence: "The evaluation results showed a top-1 accuracy of 78.3% on the test data on Imagenet."
Accuracy: 78.3

Example 4:
Expected Output:
Sentence: "Our proposed model achieved a top-1 accuracy of 74.2% when evaluated on the Imagenet dataset."
Accuracy: 74.2

Example 4:
Expected Output:
Sentence: "Our proposed model achieved a top-5 accuracy of 66.2% when evaluated on the Imagenet dataset."
Accuracy: -

Now extract the top1 accuracy of ImageNet from the following texts, {page}

Expected Output:
"""

prompt_vote_accuracy = """Find the accuracy value associated with most common sentences from the list of sentences and accuracies. Only output the accuracy value.

Example 1:
'Sentence: "ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4"\nAccuracy: 92.4',
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 92.4',
'Sentence: "ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4"\nAccuracy: 92.4',
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 74.6',
Expected Output: 92.4

Example 2:
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 74.6',
'Sentence: "ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 82.4"\nAccuracy: 82.4',
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 82.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 82.4',
Expected Output: 82.4

Example 3:
'Sentence: "It's not mentioned top-1 accuracy on ImageNet"\nAccuracy: 404',
'Sentence: "-"\nAccuracy: -',
'Sentence: "It's not mentioned top-1 accuracy on ImageNet"\nAccuracy: 404',
'Sentence: "-"\nAccuracy: 404',
Expected Output: -

Example 4:
'Sentence: "It's not mentioned."\nAccuracy: -',
'Sentence: "-"\nAccuracy: -',
'Sentence: "It's mentioned top-5 accuracy on ImageNet"\nAccuracy: -',
'Sentence: "Cocoa 23.3 21.2"\nAccuracy: 23.3',
Expected Output: -

Now extract the accuracy value associated with most common sentences, {sentences_and_accuracies}

Expected Output:
"""

prompt_vote_accuracy = """Find the accuracy value associated with most common sentences from the list of sentences and accuracies. Only output the accuracy value.

Example 1:
'Sentence: "ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4"\nAccuracy: 92.4',
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 92.4',
'Sentence: "ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4"\nAccuracy: 92.4',
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 74.6',
Expected Output: 92.4

Example 2:
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 74.6',
'Sentence: "ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 82.4"\nAccuracy: 82.4',
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 82.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 82.4',
Expected Output: 82.4

Example 3:
'Sentence: "It's not mentioned top-1 accuracy on ImageNet"\nAccuracy: 404',
'Sentence: "-"\nAccuracy: -',
'Sentence: "It's not mentioned top-1 accuracy on ImageNet"\nAccuracy: 404',
'Sentence: "-"\nAccuracy: 404',
Expected Output: -

Example 4:
'Sentence: "It's not mentioned."\nAccuracy: -',
'Sentence: "-"\nAccuracy: -',
'Sentence: "It's mentioned top-5 accuracy on ImageNet"\nAccuracy: -',
'Sentence: "Cocoa 23.3 21.2"\nAccuracy: 23.3',
Expected Output: -

Now extract the accuracy value associated with most common sentences, {sentences_and_accuracies}

Expected Output:
"""

def parse_gpt_with_page_prompt(page_with_res, prompt):
    openai_api_key = ''
    prompt2 = ChatPromptTemplate.from_template(prompt)
    output_parser = StrOutputParser()
    model = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0.0,)

    chain = prompt2 | model | output_parser
    try:
        prompt_value = chain.invoke(
            {'page': page_with_res}
        )
    except Exception as e:
        prompt_value = str(e)
    return prompt_value

def delayed_completion(delays_in_sec, **kwargs):
    time.sleep(delays_in_sec)
    return parse_gpt_with_page_prompt(**kwargs)

def parse_gpt_with_vote(sentences_and_accuracies, prompt):
    openai_api_key = ''
    model = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0.0, max_tokens=5)

    chain = ChatPromptTemplate.from_template(prompt) | model | StrOutputParser()
    try:
        prompt_value = chain.invoke(
            {'sentences_and_accuracies': sentences_and_accuracies}
        )
    except Exception as e:
        prompt_value = str(e)
    return prompt_value

##### PARTIAL PROCESS FOR SINGLE PAPER ############
idx = 11
pdf_path, rel_key, rel_page = sorted_pdf_key_page[idx]
print('expect ', groundtruth_df[['Paper Name','Model','Top-1 Accuracy']].iloc[idx].values)
rel_key = rel_key if rel_key>0 else 2
rel_pg_num = [rel_key, rel_key-1, rel_key+1]
rel_pg_num.sort()
text_dict = extract_text_from_pdf_as_dict(pdf_path)
rel_page = ''.join([texts for pg_num, texts in text_dict.items() if pg_num in rel_pg_num])

# Calculate the delay based on your rate limit
token_limit_per_minute = 10000
delay = 60.0 / token_limit_per_minute * len(rel_page) / 3

print('groundtruth resulst')
# responses = [delayed_completion(delays_in_sec=delay, page_with_res=rel_page, prompt=prompt_self_veri) for i in range(5)]
# print(responses)

# print('groundtruth resulst')
def chunk_text_keys(keys, stepsize):
    lis = []
    sublis = []
    for k in keys:
        sublis += [k]
        if k%3 == 0:
            lis.append(sublis)
            sublis = []
    return lis

# responses_lis = []
# for sublis in tqdm(chunk_text_keys(text_dict.keys(), stepsize=3)):
#     rel_page = ''.join([texts for pg_num, texts in text_dict.items() if pg_num in sublis])
#     print(sublis, len(rel_page)/4)
#     responses = [delayed_completion(delays_in_sec=delay, page_with_res=rel_page, prompt=prompt_self_veri) for i in range(5)]
#     print(responses)
#     responses_lis.append(responses)

# [parse_gpt_with_vote(res, prompt_vote_accuracy) for res in responses_lis]
##### PARTIAL PROCESS FOR SINGLE PAPER END ############
def _vote_emsemble_across_paper(idx, pdf_path, prompt_self_veri, prompt_vote_accuracy):
    # print('expect ', groundtruth_df[['Paper Name','Model','Top-1 Accuracy']].iloc[idx].values)
    text_dict = extract_text_from_pdf_as_dict(pdf_path)

    # compose sublist
    responses_lis = []
    for _, sublis in enumerate(chunk_text_keys(text_dict.keys(), stepsize=4)):
        rel_page = ''.join([texts for pg_num, texts in text_dict.items() if pg_num in sublis])
        print(sublis, len(rel_page)/4)
        responses = [delayed_completion(delays_in_sec=delay, page_with_res=rel_page, prompt=prompt_self_veri) for i in range(5)]
        responses_lis.append(responses)
        print(responses)

    # assumption is that the vote_ensemble will consistently return NA or Results
    relevant_response = [parse_gpt_with_vote(res, prompt_vote_accuracy) for res in responses_lis]
    rel_res = [ans for ans in relevant_response if ans != '404']
    final_res = parse_gpt_with_vote(rel_res, prompt_vote_accuracy) if rel_res else '404'
    print('#### EOD #####')
    return final_res

res_df_whole_paper = pd.DataFrame(data=[[idx, find_fname(pdf_path), _vote_emsemble_across_paper(idx, pdf_path, prompt_self_veri, prompt_vote_accuracy)] \
                                          for idx, (fname, pdf_path) in tqdm(sorted_pdf_key_page_20.items())], \
                                    columns = ['file_idx', 'file_name', 'gpt_vote_ensemble_whole_paper'])

reduce(lambda l, r: pd.merge(l, r, on=['file_name'],how='right'), [template, new])


  0%|                                                                                                                                                     | 0/8 [00:00<?, ?it/s]

expect  ['OmniVec: Learning robust representations with cross modal sharing'
 'OminiVec (FT)' '92.40%']
groundtruth resulst
[1, 2, 3] 3815.75


/Users/dawn.duan/Library/CloudStorage/OneDrive-CanadianTire/Documents/tetris/ivado_or/tetris-api-worker/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


['404', '404', '404', '404', '404']
[4, 5, 6] 3545.75
['Expected Output:\nSentence: "In each case, the models are pre-trained on the ImageNet [8] and then fine-tuned on the target datasets."\nAccuracy: 404', '404', 'Expected Output:\nSentence: "In each case, the models are pre-trained on the ImageNet [8] and then ﬁne-tuned on the target datasets."\nAccuracy: 404', '404', '404']
[7, 8, 9] 3841.0
['404', '404', '404', '404', '404']



 12%|█████████████████▌                                                                                                                          | 1/8 [06:36<46:18, 396.95s/it]

...


 88%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                 | 7/8 [44:45<06:16, 376.70s/it]

#### EOD #####
[1, 2, 3] 3815.5
['Expected Output:\nSentence: "Furthermore, recently proposed contemporary deep models in the ImageNet Image Classification Challenge are tested, and their effectiveness in saliency detection are investigated."\nAccuracy: 404', 'Expected Output:\nSentence: "Furthermore, recently proposed contemporary deep models in the ImageNet Image Classification Challenge are tested, and their effectiveness in saliency detection are investigated."\nAccuracy: 404', 'Expected Output:\nSentence: "Furthermore, recently proposed contemporary deep models in the ImageNet Image Classification Challenge are tested, and their effectiveness in saliency detection are investigated."\nAccuracy: 404', 'Expected Output:\nSentence: "Furthermore, recently proposed contemporary deep models in the ImageNet Image Classification Challenge are tested, and their effectiveness in saliency detection are investigated."\nAccuracy: 404', 'Expected Output:\nSentence: "Furthermore, recently propose


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [51:27<00:00, 385.99s/it]

#### EOD #####


,file_name,Paper Name,Model,Top-1 Accuracy,path
0,1512.03385v1.pdf,Deep Residual Learning for Image Recognition,ResNet-152,-,/Users/dawn.duan/Library/CloudStorage/OneDrive...
1,1703.09844v5.pdf,Multiscale Dense Networks for Resource Efficie...,MSDNet,75%,/Users/dawn.duan/Library/CloudStorage/OneDrive...
2,1803.00942v3.pdf,Not All Samples Are Created Equal: Deep Learni...,ResNet-50,-,/Users/dawn.duan/Library/CloudStorage/OneDrive...
3,1807.10108v5.pdf,Effects of Degradations on Deep Neural Network...,V-CapsNet,-,/Users/dawn.duan/Library/CloudStorage/OneDrive...
4,1807.10119v3.pdf,A Unified Approximation Framework for Compress...,AlexNet,-,/Users/dawn.duan/Library/CloudStorage/OneDrive...
5,1807.11164v1.pdf,ShuffleNet V2: Practical Guidelines for Effici...,ShuffleNet v2-50,77.20%,/Users/dawn.duan/Library/CloudStorage/OneDrive...
6,1807.11254v2.pdf,Extreme Network Compression via Filter Group A...,VGG16,77.86,/Users/dawn.duan/Library/CloudStorage/OneDrive...
7,1807.11459v1.pdf,Improving Transferability of Deep Neural Networks,ResNet-27,-,/Users/dawn.duan/Library/CloudStorage/OneDrive...
8,1807.11626v3.pdf,MnasNet: Platform-Aware Neural Architecture Se...,MnasNet,75.20%,/Users/dawn.duan/Library/CloudStorage/OneDrive...
9,1909.11155v1.pdf,Anchor Loss: Modulating Loss Scale based on Pr...,ResNet-50,76.82%,/Users/dawn.duan/Library/CloudStorage/OneDrive...


In [ ]:

reduce(lambda l, r: pd.merge(l, r, on=['file_name'],how='outer'), [template, res_df_whole_paper])

,file_name,Paper Name,Model,Top-1 Accuracy,file_idx,gpt_vote_ensemble_whole_paper
0,1512.03385v1.pdf,Deep Residual Learning for Image Recognition,ResNet-152,-,NaN,NaN
1,1703.09844v5.pdf,Multiscale Dense Networks for Resource Efficie...,MSDNet,75%,NaN,NaN
2,1803.00942v3.pdf,Not All Samples Are Created Equal: Deep Learni...,ResNet-50,-,NaN,NaN
3,1807.10108v5.pdf,Effects of Degradations on Deep Neural Network...,V-CapsNet,-,NaN,NaN
4,1807.10119v3.pdf,A Unified Approximation Framework for Compress...,AlexNet,-,NaN,NaN
5,1807.11164v1.pdf,ShuffleNet V2: Practical Guidelines for Effici...,ShuffleNet v2-50,77.20%,NaN,NaN
6,1807.11254v2.pdf,Extreme Network Compression via Filter Group A...,VGG16,77.86,NaN,NaN
7,1807.11459v1.pdf,Improving Transferability of Deep Neural Networks,ResNet-27,-,NaN,NaN
8,1807.11626v3.pdf,MnasNet: Platform-Aware Neural Architecture Se...,MnasNet,75.20%,NaN,NaN
9,1909.11155v1.pdf,Anchor Loss: Modulating Loss Scale based on Pr...,ResNet-50,76.82%,NaN,NaN


## Results with increased paper size (100)


#### Parse 100 pdfs to the destination folders

In [ ]:
import shutil
import os

# Define source and destination folder paths
source_folder = '/Users/dawn.duan/Library/CloudStorage/OneDrive-CanadianTire/Documents/tetris/ivado_or/tetris-api-worker/optimization/local_experiments/aer1810/'
destination_folder = '/Users/dawn.duan/Library/CloudStorage/OneDrive-CanadianTire/Documents/tetris/ivado_or/tetris-api-worker/optimization/local_experiments/aer1810/10_benchmark_datasets/all_papers_100/'

existing_fns = set([find_fname(pdf_path) for pdf_path in sorted_pdf_paths] + [find_fname(pdf_path) for pdf_path in sorted_pdf_paths_20])
files = os.listdir(source_folder)
pdf_files = [f for f in files if f.endswith('.pdf') if f not in existing_fns]
pdf_files

# Loop through the list of PDF files and move them
for pdf_file in pdf_files[:100]:  # Move only 100 PDF files
    source_file = os.path.join(source_folder, pdf_file)
    destination_file = os.path.join(destination_folder, pdf_file)

    # Move the file
    shutil.move(source_file, destination_file)

print(f'Moved {len(pdf_files[:100])} PDF files from {source_folder} to {destination_folder}')


Moved 100 PDF files from /Users/dawn.duan/Library/CloudStorage/OneDrive-CanadianTire/Documents/tetris/ivado_or/tetris-api-worker/optimization/local_experiments/aer1810/ to /Users/dawn.duan/Library/CloudStorage/OneDrive-CanadianTire/Documents/tetris/ivado_or/tetris-api-worker/optimization/local_experiments/aer1810/10_benchmark_datasets/all_papers_100/


## Chunk 100 pdfs

In [ ]:
prompt_self_veri = """
extract the top1 accuracy of ImageNet from the given text and return both the sentence containing the accuracy.
Answer in a number, eg. 90.2% and the accuracy value in 1 number. 404 if it's not mentioned. Use the examples below as a guide.

Example 1:
Expected Output:
Sentence: "a table showing the Top-1 and Top-5 classification accuracy using a binarized ResNet-18 on Imagenet for various ways of constructing the scaling factor. The method "Case 4: α ⊗β ⊗γ" achieved a Top-1 accuracy of 57.1."
Accuracy: 57.1

Example 2:
Expected Output:
Sentence: "In our experiments, the model reported a top-1 accuracy of 82.1% on Imagenet under the given conditions."
Accuracy: 82.1

Example 3:
Expected Output:
Sentence: "The evaluation results showed a top-1 accuracy of 78.3% on the test data on Imagenet."
Accuracy: 78.3

Example 4:
Expected Output:
Sentence: "Our proposed model achieved a top-1 accuracy of 74.2% when evaluated on the Imagenet dataset."
Accuracy: 74.2

Example 4:
Expected Output:
Sentence: "Our proposed model achieved a top-5 accuracy of 66.2% when evaluated on the Imagenet dataset."
Accuracy: -

Now extract the top1 accuracy of ImageNet from the following texts, {page}

Expected Output:
"""

prompt_vote_accuracy = """Find the accuracy value associated with most common sentences from the list of sentences and accuracies. Only output the accuracy value.

Example 1:
'Sentence: "ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4"\nAccuracy: 92.4',
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 92.4',
'Sentence: "ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4"\nAccuracy: 92.4',
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 74.6',
Expected Output: 92.4

Example 2:
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 74.6',
'Sentence: "ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 82.4"\nAccuracy: 82.4',
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 82.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 82.4',
Expected Output: 82.4

Example 3:
'Sentence: "It's not mentioned top-1 accuracy on ImageNet"\nAccuracy: 404',
'Sentence: "-"\nAccuracy: -',
'Sentence: "It's not mentioned top-1 accuracy on ImageNet"\nAccuracy: 404',
'Sentence: "-"\nAccuracy: 404',
Expected Output: -

Example 4:
'Sentence: "It's not mentioned."\nAccuracy: -',
'Sentence: "-"\nAccuracy: -',
'Sentence: "It's mentioned top-5 accuracy on ImageNet"\nAccuracy: -',
'Sentence: "Cocoa 23.3 21.2"\nAccuracy: 23.3',
Expected Output: -

Now extract the accuracy value associated with most common sentences, {sentences_and_accuracies}

Expected Output:
"""

prompt_vote_accuracy = """Find the accuracy value associated with most common sentences from the list of sentences and accuracies. Only output the accuracy value.

Example 1:
'Sentence: "ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4"\nAccuracy: 92.4',
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 92.4',
'Sentence: "ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4"\nAccuracy: 92.4',
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 74.6',
Expected Output: 92.4

Example 2:
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 92.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 74.6',
'Sentence: "ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 82.4"\nAccuracy: 82.4',
'Sentence: "SSv2 Top-1 Accuracy ViViT 65.4 68.6 80.1 85.4 ImageNet1K Top-1 Accuracy ViT 88.5 89.1 88.6 82.4 Sun RGBD Top-1 Accuracy Simple3D-former 57.3 62.4 71.4 74.6"\nAccuracy: 82.4',
Expected Output: 82.4

Example 3:
'Sentence: "It's not mentioned top-1 accuracy on ImageNet"\nAccuracy: 404',
'Sentence: "-"\nAccuracy: -',
'Sentence: "It's not mentioned top-1 accuracy on ImageNet"\nAccuracy: 404',
'Sentence: "-"\nAccuracy: 404',
Expected Output: -

Example 4:
'Sentence: "It's not mentioned."\nAccuracy: -',
'Sentence: "-"\nAccuracy: -',
'Sentence: "It's mentioned top-5 accuracy on ImageNet"\nAccuracy: -',
'Sentence: "Cocoa 23.3 21.2"\nAccuracy: 23.3',
Expected Output: -

Now extract the accuracy value associated with most common sentences, {sentences_and_accuracies}

Expected Output:
"""

def parse_gpt_with_page_prompt(page_with_res, prompt):
    openai_api_key = ''
    prompt2 = ChatPromptTemplate.from_template(prompt)
    output_parser = StrOutputParser()
    model = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0.0,)

    chain = prompt2 | model | output_parser
    try:
        prompt_value = chain.invoke(
            {'page': page_with_res}
        )
    except Exception as e:
        prompt_value = str(e)
    return prompt_value

def delayed_completion(delays_in_sec, **kwargs):
    time.sleep(delays_in_sec)
    return parse_gpt_with_page_prompt(**kwargs)

def parse_gpt_with_vote(sentences_and_accuracies, prompt):
    openai_api_key = ''
    model = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0.0, max_tokens=5)

    chain = ChatPromptTemplate.from_template(prompt) | model | StrOutputParser()
    try:
        prompt_value = chain.invoke(
            {'sentences_and_accuracies': sentences_and_accuracies}
        )
    except Exception as e:
        prompt_value = str(e)
    return prompt_value


# Calculate the delay based on your rate limit
token_limit_per_minute = 10000
delay = 60.0 / token_limit_per_minute * len(rel_page) / 3

def chunk_text_keys(keys, stepsize):
    lis = []
    sublis = []
    for k in keys:
        sublis += [k]
        if k%3 == 0:
            lis.append(sublis)
            sublis = []
    return lis

def _vote_emsemble_across_paper(idx, pdf_path, prompt_self_veri, prompt_vote_accuracy):
    # print('expect ', groundtruth_df[['Paper Name','Model','Top-1 Accuracy']].iloc[idx].values)
    text_dict = extract_text_from_pdf_as_dict(pdf_path)

    # compose sublist
    responses_lis = []
    for _, sublis in enumerate(chunk_text_keys(text_dict.keys(), stepsize=4)):
        rel_page = ''.join([texts for pg_num, texts in text_dict.items() if pg_num in sublis])
        print(sublis, len(rel_page)/4)
        responses = [delayed_completion(delays_in_sec=delay, page_with_res=rel_page, prompt=prompt_self_veri) for i in range(5)]
        responses_lis.append(responses)
        print(responses)

    # assumption is that the vote_ensemble will consistently return NA or Results
    relevant_response = [parse_gpt_with_vote(res, prompt_vote_accuracy) for res in responses_lis]
    rel_res = [ans for ans in relevant_response if ans != '404']
    final_res = parse_gpt_with_vote(rel_res, prompt_vote_accuracy) if rel_res else '404'
    print('#### EOD #####')
    return final_res

res_df_whole_paper = pd.DataFrame(data=[[idx, find_fname(pdf_path), _vote_emsemble_across_paper(idx, pdf_path, prompt_self_veri, prompt_vote_accuracy)] \
                                          for idx, (fname, pdf_path) in tqdm(sorted_pdf_key_page_100.items())], \
                                    columns = ['file_idx', 'file_name', 'gpt_vote_ensemble_whole_paper'])

reduce(lambda l, r: pd.merge(l, r, on=['file_name'],how='outer'), [template, res_df_whole_paper])

  0%|                                                                                                                                                   | 0/100 [00:00<?, ?it/s]

expect  ['OmniVec: Learning robust representations with cross modal sharing'
 'OminiVec (FT)' '92.40%']
groundtruth resulst
[1, 2, 3] 2444.25


Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


['404', '404', '404', '404', '404']
[4, 5, 6] 2535.75
['Expected Output:\nSentence: "The approach described above, with 6 scales, achieves a top-5 error rate of 13.6%."\nAccuracy: -', 'Expected Output:\nSentence: "The accurate model is more accurate than the fast one (14.18% classiﬁcation error as opposed to 16.39% in Table 2), however it requires nearly twice as many connections."\nAccuracy: 85.82', 'Expected Output:\nSentence: "The accurate model is more accurate than the fast one (14.18% classiﬁcation error as opposed to 16.39% in Table 2), however it requires nearly twice as many connections."\nAccuracy: 85.82', 'Expected Output:\nSentence: "The approach described above, with 6 scales, achieves a top-5 error rate of 13.6%."\nAccuracy: -', 'Expected Output:\nSentence: "The accurate model is more accurate than the fast one (14.18% classiﬁcation error as opposed to 16.39% in Table 2), however it requires nearly twice as many connections."\nAccuracy: 85.82']
[7, 8, 9] 1253.5
['404', '4

  1%|█▎                                                                                                                                     | 1/100 [11:09<18:24:20, 669.30s/it]

...

#### EOD #####
[1, 2, 3] 3319.0
['Expected Output:\nSentence: "We provide multiple benchmark results using various deep neural networks, such as Alexnet [19], VGG [34], DenseNet [15], as well as deep generative models, such as VAE [27]."\nAccuracy: 404', 'Expected Output:\nSentence: "We provide multiple benchmark results using various deep neural networks, such as Alexnet [19], VGG [34], DenseNet [15], as well as deep generative models, such as VAE [27]."\nAccuracy: 404', 'Expected Output:\nSentence: "We provide multiple benchmark results using various deep neural networks, such as Alexnet [19], VGG [34], DenseNet [15], as well as deep generative models, such as VAE [27]."\nAccuracy: 404', 'Expected Output:\nSentence: "We provide multiple benchmark results using various deep neural networks, such as Alexnet [19], VGG [34], DenseNet [15], as well as deep generative models, such as VAE [27]."\nAccuracy: 404', '404']
[4, 5, 6] 3433.0
['Expected Output:\nSentence: "We experiment with three

 58%|████████████████████████████████████████████████████████████████████████████▌                                                       | 58/100 [12:25:11<6:03:57, 519.94s/it]

#### EOD #####
[1, 2, 3] 2707.25
['Expected Output:\nSentence: "unlike the classiﬁcation task on the imageNet dataset, which contains tens of millions of data points, there are only about 10,000 Chinese characters."\nAccuracy: 404', 'Expected Output:\nSentence: "unlike the classiﬁcation task on the imageNet dataset, which contains tens of millions of data points, there are only about 10,000 Chinese characters."\nAccuracy: 404', 'Expected Output:\nSentence: "unlike the classiﬁcation task on the imageNet dataset, which contains tens of millions of data points, there are only about 10,000 Chinese characters."\nAccuracy: 404', 'Expected Output:\nSentence: "unlike the classiﬁcation task on the imageNet dataset, which contains tens of millions of data points, there are only about 10,000 Chinese characters."\nAccuracy: 404', 'Expected Output:\nSentence: "unlike the classiﬁcation task on the imageNet dataset, which contains tens of millions of data points, there are only about 10,000 Chinese c

 59%|█████████████████████████████████████████████████████████████████████████████▉                                                      | 59/100 [12:34:11<5:59:25, 525.98s/it]

#### EOD #####
[1, 2, 3] 823.5
['Expected Output:\nSentence: -\nAccuracy: 404', 'Expected Output:\nSentence: -\nAccuracy: 404', 'Expected Output:\nSentence: -\nAccuracy: 404', 'Expected Output:\nSentence: -\nAccuracy: 404', 'Expected Output:\nSentence: -\nAccuracy: 404']
[4, 5, 6] 1812.25
['Expected Output:\nSentence: -\nAccuracy: 404', 'Expected Output:\nSentence: -\nAccuracy: 404', 'Expected Output:\nSentence: "The method "Case 4: α ⊗β ⊗γ" achieved a Top-1 accuracy of 57.1."\nAccuracy: 57.1', 'Expected Output:\nSentence: -\nAccuracy: 404', 'Expected Output:\nSentence: "The method "Case 4: α ⊗β ⊗γ" achieved a Top-1 accuracy of 57.1."\nAccuracy: 57.1']
[7, 8, 9] 1746.5
['Expected Output:\nSentence: "AlexNet, a convolutional neural network (CNN), managed to outperform all other competing, non deep learning (we call them shallow today) methods by a large margin on the object classiﬁcation (over 1000) categories challenge."\nAccuracy: 404', 'Expected Output:\nSentence: "AlexNet, a convolu